In [1]:
from typing import NoReturn
import logging

import numpy as np
from numpy.typing import ArrayLike
import matplotlib.pyplot as plt

import scipy as sp

In [240]:
def convm(x: ArrayLike, p: int) -> np.ndarray:
    """Construct the convolution matrix of the signal x with p number of parameters.

    (N + p - 1) by p non-symmetric Toeplitz matrix
    """
    _x = np.array(x, dtype=complex).ravel()
    if p < 1:
        raise ValueError(f"{p=} must be greater or equal to 1.")

    N = len(_x) + 2 * p - 2
    # the signal centered over its support
    # needed for the signal information-preserving frequency spectrum
    xcol = (_x.copy()).reshape(-1, 1)
    logging.warning(f'\n{xcol=}')
    xpad = np.concatenate((np.zeros((p - 1, 1)), xcol, np.zeros((p - 1, 1))))
    logging.warning(f'{xpad=}\n')
    X = np.empty([len(_x) + p - 1, p], dtype=complex)
    for i in range(p):
        X[:, i] = xpad[p - i - 1:N - i, 0]
    return X

In [241]:
def covar(x: ArrayLike, p: int) -> np.ndarray:
    """Covariance Matrix.

    p x p hermitian toeplitz matrix of sample covariances
    """
    _x = np.array(x, dtype=complex)
    m = len(_x)
    # remove the mean
    _x = _x - np.mean(_x)
    conv = convm(x, p + 1)
    logging.warning(f'\n{conv.shape=}')
    logging.warning(f'\n{conv=}')
    R = conv.conjugate().transpose() @ conv / (m - 1)
    return R

In [272]:
def phd(x, p):
    Rx = sp.linalg.toeplitz(x)
    Rx = covar(x, p)
    d, v = np.linalg.eigh(Rx)
    index = np.argmin(d)
    vmin = v[:, index].copy()
    sigma = d[index]
    logging.warning(f'{v=}')
    logging.warning(f'{vmin=}')
    logging.warning(f'{sigma=}')
    rts = np.roots(vmin)
    vsig = np.delete(v, index, 1)
    phases = np.angle(rts)
    siggie = np.exp(1j * np.arange(p + 1).reshape(-1, 1) * phases)
    dtft = vsig.transpose() @ siggie.conjugate()
    dtft = np.abs(dtft)**2
    dsig = np.delete(d, index) - d[index]
    powers = np.linalg.solve(dtft, dsig)
    return sigma, vmin, powers, phases, vsig, d

In [273]:
p = 2
x = np.array([6, 1.92705 + 4.58522j, -3.42705 + 3.49541j], dtype=complex)
sigma, vmin, powers, phases, vsigs, dsig = phd(x, p)
print(f'{sigma=}')
print(f'{vmin=}')
print(f'{powers=}')
print(f'{phases=}')
print(f'{vsigs=}')
print(f'{dsig=}')

xcol=array([[ 6.     +0.j     ],
       [ 1.92705+4.58522j],
       [-3.42705+3.49541j]])
       [ 0.     +0.j     ],
       [ 6.     +0.j     ],
       [ 1.92705+4.58522j],
       [-3.42705+3.49541j],
       [ 0.     +0.j     ],
       [ 0.     +0.j     ]])

conv.shape=(5, 3)
conv=array([[ 6.     +0.j     ,  0.     +0.j     ,  0.     +0.j     ],
       [ 1.92705+4.58522j,  6.     +0.j     ,  0.     +0.j     ],
       [-3.42705+3.49541j,  1.92705+4.58522j,  6.     +0.j     ],
       [ 0.     +0.j     , -3.42705+3.49541j,  1.92705+4.58522j],
       [ 0.     +0.j     ,  0.     +0.j     , -3.42705+3.49541j]])
        -5.45023785e-01+0.00000000e+00j],
       [-2.98521271e-01-7.10624304e-01j, -2.27953641e-05+9.57514534e-06j,
        -2.46718268e-01-5.87391049e-01j],
       [-3.15349585e-01+3.21719931e-01j, -4.95005738e-01+5.04944868e-01j,
         3.81556444e-01-3.89185824e-01j]])
       -0.31534958+0.32171993j])


sigma=10.678068441422983
vmin=array([ 0.4504987 +0.j        , -0.29852127-0.7106243j ,
       -0.31534958+0.32171993j])
powers=array([63.3483118 , 63.34006828])
phases=array([1.71742813, 0.62876724])
vsigs=array([[-7.07106781e-01+0.00000000e+00j, -5.45023785e-01+0.00000000e+00j],
       [-2.27953641e-05+9.57514534e-06j, -2.46718268e-01-5.87391049e-01j],
       [-4.95005738e-01+5.04944868e-01j,  3.81556444e-01-3.89185824e-01j]])
dsig=array([10.67806844, 27.66469226, 88.70772968])
